In [51]:
import dataclasses

from joshpyutil import mpl
import numpy as np
import scipy.interpolate
import progressbar


t_max = 3.
t_rain_stop = 3
lax_friedrichs_dt = 0.005
n = 200
terrain_r = 0.2
terrain_h = 0.3
gamma = 0.1
R = 0.01
g = 4

x, dx = np.linspace(0, 1, n, retstep=True)
y, dy = x, dx
x_mid = (x[1:] + x[:-1]) / 2
y_mid = x_mid

xx = x[:, np.newaxis]
yy = y[np.newaxis, :]


def grad(field):
    spline = scipy.interpolate.RectBivariateSpline(x, y, field, kx=3, ky=3)
    return np.stack([spline.partial_derivative(dx=1, dy=0)(x, y), spline.partial_derivative(dx=0, dy=1)(x, y)], axis=2)


terrain = (
    # 1.5 * terrain_h * np.exp(-((xx - 1/2)**2 + (yy - 1/2)**2) / terrain_r**2)
    + 1.5 * terrain_h * xx
    + 0.3 * terrain_h * np.sin(4 * np.pi * (xx - 1/2) + 1) * np.sin(9 * np.pi * (yy - 1/3))
)
grad_terrain = grad(terrain)
water = np.zeros(n)
moment = np.zeros(n)


@dataclasses.dataclass
class Fields:
    # 0: x, 1: y
    height: np.ndarray[float]
    # 0: x, 1: y, 2: component
    momentum: np.ndarray[float]

    def pack(self) -> np.ndarray[float]:
        stacked = np.stack([self.height, self.momentum[:, :, 0], self.momentum[:, :, 1]], axis=2)
        return np.ravel(stacked)

    @classmethod
    def unpack(cls, packed: np.ndarray[float]) -> 'Fields':
        stacked = packed.reshape((n, n, 3))
        return cls(height=stacked[:, :, 0], momentum=stacked[:, :, 1:3])

    @classmethod
    def zeros(cls) -> 'Fields':
        return cls(height=np.zeros((n, n)), momentum=np.zeros((n, n, 2)))


def lax_friedrichs_diff(field: np.ndarray[float]) -> np.ndarray[float]:
    field_2nd_deriv_x = field[2:, 1:-1] + field[:-2, 1:-1] - 2 * field[1:-1, 1:-1]
    field_2nd_deriv_y = field[1:-1, 2:] + field[1:-1, :-2] - 2 * field[1:-1, 1:-1]
    return 1 / 2 / lax_friedrichs_dt * (field_2nd_deriv_x + field_2nd_deriv_y)


#   0   1        n-1
# |-*-|-*-|.....|-*-|
# 0   1   2    n-1  n


def interpolate_mid(field: np.ndarray[float], axis: int) -> np.ndarray[float]:
    slice_arg = (slice(None),) * axis
    return (field[*(slice_arg + (slice(1, None),))] + field[*(slice_arg + (slice(None, -1),))]) / 2

pbar = progressbar.ProgressBar(max_value=t_max)

def time_deriv_func(t, packed) -> np.ndarray:
    pbar.update(round(t * 1e2) / 1e2)
    
    fields = Fields.unpack(packed)
    
    grad_height = grad(fields.height)
    accel_grav = -g / (1 + np.sum((grad_height + grad_terrain)**2, axis=2))[:, :, np.newaxis] * (grad_height + grad_terrain)
    
    height_flux_x = interpolate_mid(fields.momentum.copy(), axis=0)
    height_flux_y = interpolate_mid(fields.momentum.copy(), axis=1)
    
    velocity = fields.momentum.copy()
    tiny_mask = fields.height < 0.001
    velocity[tiny_mask] = 0
    velocity[~tiny_mask] /= fields.height[~tiny_mask][:, np.newaxis]
    momentum_flux_centers = fields.momentum[:, :, :, np.newaxis] * velocity[:, :, np.newaxis, :]
    
    momentum_flux_x = interpolate_mid(momentum_flux_centers, axis=0)
    momentum_flux_y = interpolate_mid(momentum_flux_centers, axis=1)
    
    fields_time_deriv = Fields.zeros()
    
    if t < t_rain_stop:
        rain_rate = R
    else:
        rain_rate = 0
    
    # $\delta x \delta y (dh/dt) = \delta y (p_x(x+\delta x/2,y) - p_x(x-\delta x/2,y)) + \delta x (p_y(x,y+\delta y) - p_y(x,y-\delta y))
    fields_time_deriv.height[1:-1, :] += -1/dx * (height_flux_x[1:, :, 0] - height_flux_x[:-1, :, 0])
    fields_time_deriv.height[:, 1:-1] += -1/dy * (height_flux_y[:, 1:, 1] - height_flux_y[:, :-1, 1])
    fields_time_deriv.height[1:-1, 1:-1] += lax_friedrichs_diff(fields.height)
    fields_time_deriv.height += rain_rate #* np.exp(-((xx - 1/2) / terrain_r)**2 - ((yy - 1/2) / terrain_r)**2)
        # * np.sin((np.pi * x[1:-1] + np.pi * t) / terrain_r)**2
    # fields_time_deriv.height[0] = fields_time_deriv.height[1]
    # fields_time_deriv.height[-1] = fields_time_deriv.height[-2]
    # fields_time_deriv.height[0] = water_wave_speed * grad_height[0]
    # fields_time_deriv.height[-1] = -water_wave_speed * grad_height[-1]
    
    negative_mask = fields.height < -0.0001
    fields_time_deriv.height[negative_mask] = -fields.height[negative_mask]
    
    fields_time_deriv.momentum[1:-1, :] += -1/dx * (momentum_flux_x[1:, :, 0] - momentum_flux_x[:-1, :, 0])
    fields_time_deriv.momentum[:, 1:-1] += -1/dy * (momentum_flux_y[:, 1:, 1] - momentum_flux_y[:, :-1, 1])
    fields_time_deriv.momentum[1:-1, 1:-1] += accel_grav[1:-1, 1:-1] * fields.height[1:-1, 1:-1, np.newaxis]
    fields_time_deriv.momentum += -gamma * fields.momentum
    fields_time_deriv.momentum[1:-1, 1:-1] += lax_friedrichs_diff(fields.momentum)
    # fields_time_deriv.momentum[0] = fields_time_deriv.momentum[1]
    # fields_time_deriv.momentum[-1] = fields_time_deriv.momentum[-2]
    
    return fields_time_deriv.pack()


print('Solving... ', flush=True)
init_fields = Fields.zeros()
# slice_ = slice(10 * n // 20, 14 * n // 20)
# init_fields.height[slice_, slice_] = terrain.max() + 0.5 - terrain[slice_, slice_]
sol = scipy.integrate.solve_ivp(
    time_deriv_func,
    (0, t_max),
    init_fields.pack(),
    # method='LSODA',
    # method='BDF',
    method='RK23',
    # method='DOP853',
    t_eval=np.linspace(0, t_max, 300),
)

sol_interp = scipy.interpolate.interp1d(sol.t, sol.y.T, axis=0)

print(f'Done, num steps: {sol.t.shape[0]}', flush=True)

Solving... 


100% (3.0 of 3.0) |######################| Elapsed Time: 0:00:50 ETA:  00:00:00

Done, num steps: 300


In [63]:
color_map([[0, 1, 100]])

array([[[0.        , 0.        , 0.        , 1.        ],
        [0.00261345, 0.        , 0.16869201, 1.        ],
        [0.23996741, 0.56518361, 0.3660476 , 1.        ]]])

In [78]:
from matplotlib import colormaps

height_scale = Fields.unpack(sol_interp(t_max)).height.max()
color_map = colormaps.get_cmap('gist_earth')

frame_rate_hz = 20
num_frames = 150
with mpl.autovideo('video_2d.mp4', subplot_kw={"projection": "3d"}, frame_rate_hz=frame_rate_hz, size_inches=(6, 6)) as av:
    for i in progressbar.progressbar(range(num_frames)):
        t = i / num_frames * t_max
        fields = Fields.unpack(sol_interp(t))
        
        facecolors = color_map(((height_scale - fields.height)/height_scale)**40 + 0.2)
        
        # facecolors = np.empty(fields.height.shape, dtype=object)
        # terrain_mask = fields.height < 0.005
        # facecolors[terrain_mask] = 'sienna'
        # facecolors[~terrain_mask] = 'dodgerblue'
        
        with av.next_frame() as ap:
            # _ = ap.ax.plot_surface(xx, yy, terrain, linewidth=0, color='sienna')
            _ = ap.ax.plot_surface(xx, yy, terrain + fields.height, linewidth=0, facecolors=facecolors, rcount=100, ccount=100)
            _ = ap.ax.set_axis_off()
            _ = ap.ax.set_zlim(terrain.min(), 1)
            _ = ap.ax.view_init(elev=40., azim=90 + 30 * i/num_frames)
            
            # _ = ap.ax.fill_between(x, min_terrain, terrain[n//2], label='terrain', color='sienna')
            # _ = ap.ax.fill_between(x, terrain[n//2], terrain[n//2] + fields.height[n//2], label='water', color='dodgerblue')





# with mpl.autoplot(subplot_kw={"projection": "3d"}) as ap:
#     fields = Fields.unpack(sol_interp(1))
#     height = fields.height
#     _ = ap.ax.plot_surface(xx, yy, terrain, cmap=cm.gist_earth)
#     _ = ap.ax.plot_surface(xx, yy, terrain + fields.height, cmap=cm.ocean, alpha=0.5)
#     _ = ap.ax.set_axis_off()

100% (150 of 150) |######################| Elapsed Time: 0:02:35 Time:  0:02:35


In [24]:
frame_rate_hz = 20
num_frames = 150
with mpl.autovideo('video_2d.mp4', 3, frame_rate_hz=frame_rate_hz, size_inches=(6, 6), sharex=True) as av:
    for i in progressbar.progressbar(range(num_frames)):
        t = i / num_frames * t_max
        fields = Fields.unpack(sol_interp(t))
        fields_time_deriv = Fields.unpack(time_deriv_func(t, fields.pack()))
        with av.next_frame() as ap:
            # ap.plot(x, fields.height)
            min_terrain = terrain.min()
            _ = ap.ax.fill_between(x, min_terrain, terrain[n//2], label='terrain', color='sienna')
            _ = ap.ax.fill_between(x, terrain[n//2], terrain[n//2] + fields.height[n//2], label='water', color='dodgerblue')
            # if t < t_rain_stop:
            #     _ = ap.ax.fill_between(x, terrain + fields.height, terrain.max() + 0.1, label='water', color='lightgray')
            ap.set(xlim=[0, 1], ylim=[None, 0.3])
            
            ap = ap.next()
            _ = ap.ax.fill_between(y, min_terrain, terrain[:, n//2], label='terrain', color='sienna')
            _ = ap.ax.fill_between(y, terrain[:, n//2], terrain[:, n//2] + fields.height[:, n//2], label='water', color='dodgerblue')
            # if t < t_rain_stop:
            #     _ = ap.ax.fill_between(x, terrain + fields.height, terrain.max() + 0.1, label='water', color='lightgray')
            ap.set(xlim=[0, 1], ylim=[None, 0.3])

#             ap = ap.next()
#             ap.plot(x, fields.momentum, label='$p$ (momentum density)')
#             # ap.plot(x, fields.momentum / (fields.height + 1e-9), label='v')
#             ap.set(ylim=[-.075, .075])
#             ap.legend()
            
#             ap = ap.next()
#             ap.plot(x, fields_time_deriv.momentum, label='$dp/dt$', color=mpl.COLORS[3])
#             ap.set(ylim=[-.3, .3], xlabel='$x$')
#             ap.legend()

100% (150 of 150) |######################| Elapsed Time: 0:00:49 Time:  0:00:49
